# YAML File Comparison

This notebook demonstrates how to compare two YAML files using the `yaml_shredder` package.

The comparison includes:

- **Schema comparison**: Detect structural differences (tables, columns)
- **Data comparison**: Identify added, removed, and modified records using primary key detection


## Setup

Import the required modules from yaml_shredder.


In [12]:
from pathlib import Path
from yaml_shredder import YAMLComparator, DataComparer, TableGenerator
import yaml

## Create Sample YAML Files for Comparison

Let's create two sample YAML files to demonstrate the comparison features.


In [13]:
# Create a temporary directory for our sample files
import tempfile
temp_dir = Path(tempfile.mkdtemp())

# Sample YAML 1 - Original configuration
yaml1_content = """
version: "1.0"
environment: production

users:
  - id: 1
    name: alice
    email: alice@example.com
    role: admin
  - id: 2
    name: bob
    email: bob@example.com
    role: user

servers:
  - name: web-01
    ip: 192.168.1.10
    port: 8080
  - name: web-02
    ip: 192.168.1.11
    port: 8080

database:
  host: localhost
  port: 5432
  name: myapp_db
"""

# Sample YAML 2 - Modified configuration
yaml2_content = """
version: "1.1"
environment: production

users:
  - id: 1
    name: alice
    email: alice@newdomain.com
    role: admin
  - id: 2
    name: bob
    email: bob@example.com
    role: moderator
  - id: 3
    name: charlie
    email: charlie@example.com
    role: user

servers:
  - name: web-01
    ip: 192.168.1.10
    port: 8080
  - name: web-03
    ip: 192.168.1.12
    port: 9090

database:
  host: db.example.com
  port: 5432
  name: myapp_db
  replicas: 3
"""

# Write the YAML files
yaml1_path = temp_dir / "config_v1.yaml"
yaml2_path = temp_dir / "config_v2.yaml"

yaml1_path.write_text(yaml1_content)
yaml2_path.write_text(yaml2_content)

print(f"Created: {yaml1_path}")
print(f"Created: {yaml2_path}")

Created: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/config_v1.yaml
Created: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/config_v2.yaml


## Initialize the YAMLComparator


In [14]:
# Create a comparator with a temporary output directory for SQLite databases
comparator = YAMLComparator(output_dir=temp_dir / "dbs")
print(f"Comparator initialized with output dir: {comparator.output_dir}")

Comparator initialized with output dir: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs


## Option 1: Schema Comparison Only

Compare the structure of two YAML files (tables, columns, row counts).


In [15]:
# Schema comparison (original method)
schema_report = comparator.compare_yaml_files(
    yaml1_path,
    yaml2_path,
    keep_dbs=True  # Keep databases for inspection
)

print(schema_report)

Connected to SQLite database: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v1.db
  Loaded 1 rows into table: CONFIG_V1
  Loaded 1 rows into table: DATABASE
  Loaded 2 rows into table: USERS
  Loaded 2 rows into table: SERVERS

✓ Loaded 4 tables into /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v1.db
Database connection closed
Connected to SQLite database: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v2.db
  Loaded 1 rows into table: CONFIG_V2
  Loaded 1 rows into table: DATABASE
  Loaded 3 rows into table: USERS
  Loaded 2 rows into table: SERVERS

✓ Loaded 4 tables into /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v2.db
Database connection closed
# YAML Comparison Report

## Summary

- **File 1:** config_v1
- **File 2:** config_v2
- **Tables in common:** 3
- **Tables only in File 1:** 1
- **Tables only in File 2:** 1
- **Schema differences:** 1
- **Row count differences:** 1

##

## Option 2: Data Comparison with Primary Key Detection

Compare the actual data between YAML files, detecting:

- Added records
- Removed records
- Modified records (with field-level changes)


In [16]:
# Data comparison with primary key detection
data_result = comparator.compare_data(
    yaml1_path,
    yaml2_path
)

# Print summary
print("=" * 60)
print("DATA COMPARISON SUMMARY")
print("=" * 60)
print(f"\nTables matched: {data_result['summary']['tables_matched']}")
print(f"Tables only in first file: {data_result['summary']['tables_only_in_first']}")
print(f"Tables only in second file: {data_result['summary']['tables_only_in_second']}")
print(f"Tables with differences: {data_result['summary']['tables_with_differences']}")

No primary key detected for CONFIG_V1, using row-based comparison


DATA COMPARISON SUMMARY

Tables matched: 4
Tables only in first file: 0
Tables only in second file: 0
Tables with differences: 4


## Explore Table-Level Comparisons


In [17]:
# Detailed comparison for each table
for table_comp in data_result['table_comparisons']:
    print(f"\n{'=' * 60}")
    print(f"Table: {table_comp['table_name']}")
    print(f"{'=' * 60}")

    # Primary key detection
    pk = table_comp.get('primary_key', [])
    pk_detected = table_comp.get('primary_key_detected', False)
    print(f"Primary Key: {pk if pk else 'Not detected'}")

    # Row statistics
    print(f"\nRow Statistics:")
    print(f"  - Rows in first file: {table_comp.get('rows_in_first', 0)}")
    print(f"  - Rows in second file: {table_comp.get('rows_in_second', 0)}")
    print(f"  - Rows added: {table_comp.get('rows_only_in_second', 0)}")
    print(f"  - Rows removed: {table_comp.get('rows_only_in_first', 0)}")
    print(f"  - Rows modified: {table_comp.get('rows_modified', 0)}")
    print(f"  - Rows unchanged: {table_comp.get('rows_unchanged', 0)}")

    # Show field differences if any
    if table_comp.get('field_differences'):
        print(f"\nField-level changes:")
        for diff in table_comp['field_differences'][:5]:  # Show first 5
            pk_str = ", ".join(f"{k}={v}" for k, v in diff['primary_key'].items())
            print(f"  [{pk_str}] {diff['field']}: '{diff['old_value']}' → '{diff['new_value']}'")


Table: DATABASE
Primary Key: ['name']

Row Statistics:
  - Rows in first file: 1
  - Rows in second file: 1
  - Rows added: 0
  - Rows removed: 0
  - Rows modified: 1
  - Rows unchanged: 0

Field-level changes:
  [name=myapp_db] host: 'localhost' → 'db.example.com'

Table: SERVERS
Primary Key: ['name']

Row Statistics:
  - Rows in first file: 2
  - Rows in second file: 2
  - Rows added: 1
  - Rows removed: 1
  - Rows modified: 0
  - Rows unchanged: 1

Table: USERS
Primary Key: ['id']

Row Statistics:
  - Rows in first file: 2
  - Rows in second file: 3
  - Rows added: 1
  - Rows removed: 0
  - Rows modified: 2
  - Rows unchanged: 0

Field-level changes:
  [id=1] email: 'alice@example.com' → 'alice@newdomain.com'
  [id=2] role: 'user' → 'moderator'

Table: CONFIG_V1
Primary Key: Not detected

Row Statistics:
  - Rows in first file: 1
  - Rows in second file: 1
  - Rows added: 1
  - Rows removed: 1
  - Rows modified: 0
  - Rows unchanged: 0


## Option 3: Full Comparison (Schema + Data)

Get both schema and data comparison in one call.


In [18]:
# Full comparison combining schema and data analysis
report_path = temp_dir / "full_comparison_report.md"

schema_report, data_result = comparator.compare_yaml_files_full(
    yaml1_path,
    yaml2_path,
    output_report=report_path
)

print(f"Report saved to: {report_path}")
print(f"\nReport preview (first 2000 chars):")
print("-" * 60)
print(report_path.read_text()[:2000])

No primary key detected for CONFIG_V1, using row-based comparison


Connected to SQLite database: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v1.db
  Loaded 1 rows into table: CONFIG_V1
  Loaded 1 rows into table: DATABASE
  Loaded 2 rows into table: USERS
  Loaded 2 rows into table: SERVERS

✓ Loaded 4 tables into /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v1.db
Database connection closed
Connected to SQLite database: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v2.db
  Loaded 1 rows into table: CONFIG_V2
  Loaded 1 rows into table: DATABASE
  Loaded 3 rows into table: USERS
  Loaded 2 rows into table: SERVERS

✓ Loaded 4 tables into /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/dbs/config_v2.db
Database connection closed
Report saved to: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmpc68m2ceg/full_comparison_report.md

Report preview (first 2000 chars):
------------------------------------------------------------
# YAML Comparison Report

## Summar

## Generate Data Comparison Report


In [19]:
# Generate a markdown report from data comparison results
data_comparer = DataComparer()
data_report = data_comparer.generate_comparison_report(data_result)

print(data_report)

# Data Comparison Report

## Summary

- **Tables matched:** 4
- **Tables only in first file:** 0
- **Tables only in second file:** 0
- **Tables with differences:** 4

## Table Comparisons

### Table: `DATABASE`

**Primary Key:** `name`

**Row Statistics:**

| Metric | Count |
|--------|-------|
| Rows in first file | 1 |
| Rows in second file | 1 |
| Rows only in first | 0 |
| Rows only in second | 0 |
| Rows modified | 1 |
| Rows unchanged | 0 |

**Columns only in second file:**
- `replicas`

**Field-level differences:**

| Primary Key | Field | Old Value | New Value |
|-------------|-------|-----------|-----------|
| name=myapp_db | `host` | localhost | db.example.com |

---

### Table: `SERVERS`

**Primary Key:** `name`

**Row Statistics:**

| Metric | Count |
|--------|-------|
| Rows in first file | 2 |
| Rows in second file | 2 |
| Rows only in first | 1 |
| Rows only in second | 1 |
| Rows modified | 0 |
| Rows unchanged | 1 |

---

### Table: `USERS`

**Primary Key:** `id`

**R

## Advanced: Specify Primary Keys Explicitly

If automatic primary key detection doesn't work for your data, you can specify keys explicitly.


In [20]:
# Specify primary keys explicitly for certain tables
explicit_pks = {
    "USERS": ["id"],
    "SERVERS": ["name"],
}

data_result_explicit = comparator.compare_data(
    yaml1_path,
    yaml2_path,
    primary_keys=explicit_pks
)

# Check that explicit keys were used
for table_comp in data_result_explicit['table_comparisons']:
    print(f"{table_comp['table_name']}: PK = {table_comp.get('primary_key', 'N/A')}")

No primary key detected for CONFIG_V1, using row-based comparison


DATABASE: PK = ['name']
SERVERS: PK = ['name']
USERS: PK = ['id']
CONFIG_V1: PK = []


## Compare Your Own YAML Files

Replace the paths below with your actual YAML files.


In [21]:
# Uncomment and modify the paths to compare your own files
my_yaml1 = Path("/Users/igladyshev/PythonWithUV/generic-reporting/resources/master-mpm/DC/DC_005-mpm.yaml")
my_yaml2 = Path("/Users/igladyshev/PythonWithUV/generic-reporting/resources/master-mpm/CO/CO_005-mpm.yaml")

comparator = YAMLComparator()
schema_report, data_result = comparator.compare_yaml_files_full(
    my_yaml1,
    my_yaml2,
    output_report="/Users/igladyshev/PythonWithUV/schema-sentinel/resources/metadata-doc/DC-005-to-CO-005-mpm-comparison.md"
)

Connected to SQLite database: temp_dbs/DC_005-mpm.db
  Loaded 1 rows into table: DEPLOYMENT
  Loaded 51 rows into table: COMMUNITIES
  Loaded 121 rows into table: ACTIONS

✓ Loaded 3 tables into temp_dbs/DC_005-mpm.db
Database connection closed
Connected to SQLite database: temp_dbs/CO_005-mpm.db
  Loaded 1 rows into table: DEPLOYMENT
  Loaded 4 rows into table: COMMUNITIES
  Loaded 132 rows into table: ACTIONS

✓ Loaded 3 tables into temp_dbs/CO_005-mpm.db
Database connection closed


## YAML Sync — Validate, Compare & Merge

`sync_yaml_files` combines schema validation, node-level discrepancy detection, and optional merge into a single call.

Key features:

- **Dynamic identifier detection** — list items are identified by a naturally unique key (e.g. `action_code`, `name`) instead of positional index, so paths like `$.actions[TEMP_MONITOR].parents[0]` tell you exactly which record changed.
- **Merge directions** — `none` (report only), `left-to-right`, `right-to-left`, or `both`.


In [ ]:
# ── Sync with sample data (report-only mode) ──────────────────────────
# Re-create temp dir and comparator for this section
import tempfile
sync_temp = Path(tempfile.mkdtemp())

left_yaml = sync_temp / "left.yaml"
right_yaml = sync_temp / "right.yaml"

left_yaml.write_text("""
root:
  version: "1.0"
  actions:
    - action_code: TEMP_MONITOR
      schedule_crontab: "0 8 * * *"
      parents:
        - etl.ROSI.player
        - etl.ROSI.businessactivity
    - action_code: DAILY_SUMMARY
      schedule_crontab: "0 12 * * *"
      parents:
        - etl.ROSI.summary
  servers:
    - name: web-01
      ip: 10.0.0.1
    - name: web-02
      ip: 10.0.0.2
""")

right_yaml.write_text("""
root:
  version: "1.1"
  actions:
    - action_code: TEMP_MONITOR
      schedule_crontab: "0 9 * * *"
      parents:
        - player
        - etl.ROSI.businessactivity
    - action_code: DAILY_SUMMARY
      schedule_crontab: "0 12 * * *"
      parents:
        - etl.ROSI.summary
  servers:
    - name: web-01
      ip: 10.0.0.99
    - name: web-02
      ip: 10.0.0.2
""")

sync_comparator = YAMLComparator(output_dir=sync_temp / "dbs")

# Save report to a stable location inside the project
sync_report_path = Path("../resources/metadata-doc/sample-sync-report.md").resolve()

result = sync_comparator.sync_yaml_files(
    left_file=left_yaml,
    right_file=right_yaml,
    output_report=sync_report_path,
    merge_direction="none",        # report-only — no files modified
    root_table_name="root",
)

# ── Show discrepancy paths ────────────────────────────────────────────
print("=== Discrepancy Paths ===\n")
for category, items in result["discrepancies"].items():
    if items:
        print(f"  {category}:")
        for item in items:
            print(f"    {item['path']}")

print(f"\n📄 Report saved to: {sync_report_path}")


Connected to SQLite database: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmperpjg7_7/dbs/left.db
  Loaded 1 rows into table: ROOT
  Loaded 2 rows into table: ROOT_ACTIONS
  Loaded 2 rows into table: ROOT_SERVERS

✓ Loaded 3 tables into /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmperpjg7_7/dbs/left.db
Database connection closed
Connected to SQLite database: /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmperpjg7_7/dbs/right.db
  Loaded 1 rows into table: ROOT
  Loaded 2 rows into table: ROOT_ACTIONS
  Loaded 2 rows into table: ROOT_SERVERS

✓ Loaded 3 tables into /var/folders/nk/zxv2pgts2fjc1mxblnjmp7y00000gq/T/tmperpjg7_7/dbs/right.db
Database connection closed
=== Discrepancy Paths ===

  different_values:
    $.root.actions[TEMP_MONITOR].parents[0]
    $.root.actions[TEMP_MONITOR].schedule_crontab
    $.root.servers[web-01].ip
    $.root.version

📄 Report saved to: /Users/igladyshev/PythonWithUV/schema-sentinel/resources/metadata-doc/sample-sync-report.md


### Sync with your own YAML files

Uncomment and adjust the paths below. Set `merge_direction` to `"left-to-right"` to push left values into the right file (or `"right-to-left"` / `"both"`).


In [ ]:
# # ── Sync your own files ─────────────────────────────────────────────
# my_left  = Path("/Users/igladyshev/PythonWithUV/generic-reporting/resources/master-mpm/DC/DC_005-mpm.yaml")
# my_right = Path("/Users/igladyshev/PythonWithUV/generic-reporting/resources/master-mpm/CO/CO_005-mpm.yaml")
#
# sync_cmp = YAMLComparator()
# my_result = sync_cmp.sync_yaml_files(
#     left_file=my_left,
#     right_file=my_right,
#     output_report=Path("resources/metadata-doc/DC-005-to-CO-005-sync-report.md"),
#     merge_direction="none",          # change to "left-to-right" etc. when ready
#     root_table_name="root",
# )
# print(my_result["report"][:3000])


## Cleanup


In [24]:
# Clean up temporary files
import shutil
shutil.rmtree(temp_dir, ignore_errors=True)
print("Temporary files cleaned up.")

Temporary files cleaned up.
